In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.probability import FreqDist
import string
from wordcloud import WordCloud
from konlpy.tag import Okt
from collections import Counter
import ast

import platform
from matplotlib import font_manager, rc

plt.rcParams['axes.unicode_minus']=False

path = 'c:/Windows/Fonts/malgun.ttf'
font_name = font_manager.FontProperties(fname=path).get_name()
rc('font',family=font_name)

In [28]:
cnam = pd.read_csv('data/전처리 데이터/충남1.csv')
one = pd.read_csv('data/전처리 데이터/강원1.csv')
gg = pd.read_csv('data/전처리 데이터/경기1.csv')
ic = pd.read_csv('data/전처리 데이터/인천1.csv')
cbook = pd.read_csv('data/전처리 데이터/충북1.csv')
jeju = pd.read_csv('data/전처리 데이터/제주도1.csv')
seoul = pd.read_csv('data/전처리 데이터/서울1.csv')
gnam = pd.read_csv('data/전처리 데이터/경남1.csv')
gbook = pd.read_csv('data/전처리 데이터/경북1.csv')
daegu = pd.read_csv('data/전처리 데이터/대구1.csv')
daej = pd.read_csv('data/전처리 데이터/대전1.csv')
busan = pd.read_csv('data/전처리 데이터/부산1.csv')
sejong = pd.read_csv('data/전처리 데이터/세종1.csv')
ulsan = pd.read_csv('data/전처리 데이터/울산1.csv')
jnam = pd.read_csv('data/전처리 데이터/전남1.csv')
jbook = pd.read_csv('data/전처리 데이터/전북1.csv')
gj = pd.read_csv('data/전처리 데이터/광주1.csv')

## 결측치 처리 및 형태 변경, 정렬

In [17]:
# 1. 딕셔너리에 문자열이 덮혀있어서 제거
def dict(df):
    for i in range(len(df)):
        df.at[i, 'good_keyword_ratio'] = ast.literal_eval(df.at[i, 'good_keyword_ratio'])
        df.at[i, 'services'] = ast.literal_eval(df.at[i, 'services'])
        df.at[i, 'star&reviews'] = ast.literal_eval(df.at[i, 'star&reviews'])
        df.at[i, 'good_keyword_freq'] = ast.literal_eval(df.at[i, 'good_keyword_freq'])
        df.at[i, 'good_keyword'] = ast.literal_eval(df.at[i, 'good_keyword'])
        df.at[i, 'bad_keyword'] = ast.literal_eval(df.at[i, 'bad_keyword'])        
    return df 

In [4]:
# 2. 리뷰 높은순 정렬
def review_sort(df):
    for i in range(len(df)):
        reviews = df.loc[i, 'star&reviews']
        # NaN인 경우 빈 리스트로 처리
        if isinstance(reviews, float) and np.isnan(reviews):
            sorted_reviews = []
        else:
            sorted_reviews = sorted(reviews, key=lambda x: x[0], reverse=True)
        df.loc[i, 'star&reviews'] = str(sorted_reviews)
        df.at[i, 'star&reviews'] = ast.literal_eval(df.at[i, 'star&reviews'])
    return df

In [5]:
# 3. 키워드 빈도수 정리
def count_keyword(df, keywordlist):
    df['good_keyword_freq'] = pd.Series(dtype=str)
    for i in range(len(df)):
        keyword_freq = {}
        for keyword in keywordlist:
            try:
                if df.loc[i, 'good'].count(keyword):
                    freq = df.loc[i,'good'].count(keyword)
                    keyword_freq[keyword] = freq
            except:
                keyword_freq[keyword] = '0'
        df.loc[i,'good_keyword_freq'] = str(keyword_freq)

In [29]:
# 'services'에 nan값을 []로 변경
lists = cnam, one, gg, ic, cbook, jeju, seoul, gnam, gbook, daegu,daej, busan, sejong, ulsan, jnam, jbook, gj
for idx, df in enumerate(lists):
    lists[idx]['services'] = df['services'].fillna("[]")
cnam, one, gg, ic, cbook, jeju, seoul, gnam, gbook, daegu,daej, busan, sejong, ulsan, jnam, jbook, gj = lists

In [30]:
# 지역별 긍정 키워드 TOP10
one_keywordlist = '청결 객실 시설 바다 직원 속초 이용 가격 예약 위치'.split()
cnam_keywordlist = '청결 시설 객실 이용 직원 주변 조식 가격 화장실 주차'.split()
gg_keywordlist = '청결 객실 직원 이용 시설 주변 주차 침대 조식 가격'.split()
ic_keywordlist = '청결 객실 시설 직원 이용 주변 가격 주차 체크 근처'.split()
cbook_keywordlist = '청결 객실 시설 직원 이용 주변 조식 청주 예약 생각'.split()
jeju_keywordlist = '청결 직원 가격 객실 시설 이용 침대 위치 근처 주차'.split()
seoul_keywordlist = '청결 객실 직원 이용 위치 가격 시설 체크 조식 근처'.split()
gnam_keywordlist = '청결 객실 시설 직원 이용 조식 예약 주변 바다 침대'.split()
gbook_keywordlist = '청결 객실 시설 조식 직원 경주 이용 가격 위치 가족'.split()
daegu_keywordlist = '청결 조식 시설 객실 직원 이용 주변 주차 가격 예약'.split()
daej_keywordlist = '청결 객실 가격 조식 주차 이용 직원 시설 위치 주변'.split()
busan_keywordlist = '청결 객실 직원 위치 바다 이용 가격 시설 해운대 오션'.split()
sejong_keywordlist = '청결 시설 여유 주차 식당 상태 주변 청소 침구 주차공간'.split()
ulsan_keywordlist = '청결 객실 주차 시설 위치 직원 조식 가격 침대 이용'.split()
jnam_keywordlist = '청결 객실 시설 직원 여수 조식 가격 침대 이용 바다'.split()
jbook_keywordlist = '청결 직원 객실 시설 전주 조식 마을 이용 침대 위치'.split()
gj_keywordlist = '청결 조식 시설 직원 객실 이용 주차 침대 가격 주변'.split()

In [31]:
# 키워드 빈도수 세기 모든 지역 진행
lists = [cnam, one, gg, ic, cbook, jeju, seoul, gnam, gbook, daegu,daej, busan, sejong, ulsan, jnam, jbook, gj]
keyword_lists =[cnam_keywordlist, one_keywordlist, gg_keywordlist, ic_keywordlist, cbook_keywordlist, jeju_keywordlist, seoul_keywordlist, gnam_keywordlist, gbook_keywordlist,daegu_keywordlist, daej_keywordlist, busan_keywordlist, sejong_keywordlist, ulsan_keywordlist,jnam_keywordlist, jbook_keywordlist, gj_keywordlist]

keyword_dict = {}
for idx, df in enumerate(lists):
    count_keyword(lists[idx], keyword_lists[idx])

cnam, one, gg, ic, cbook, jeju, seoul, gnam, gbook, daegu,daej, busan, sejong, ulsan, jnam, jbook, gj = lists

In [32]:
# 모든 지역 진행
lists = cnam, one, gg, ic, cbook, jeju, seoul, gnam, gbook, daegu,daej, busan, sejong, ulsan, jnam, jbook, gj

for i in lists:
    try:
        dict(i)
        review_sort(i)
    except (SyntaxError, ValueError):
        pass
cnam, one, gg, ic, cbook, jeju, seoul, gnam, gbook, daegu,daej, busan, sejong, ulsan, jnam, jbook, gj = lists

In [20]:
cnam

,region,hotel_name,services,location,star&reviews,good,bad,good_keyword,bad_keyword,good_keyword_ratio,위도,경도,good_keyword_freq
0,충남,레이크힐 호텔,"[조식포함, 와이파이, 욕실용품, 금연, TV, 엘레베이터, 에어컨, 냉장고, 객실...",충남 논산시 가야곡면 종연리 148-9,"[[5.0, 3일 전, 시설도 좋고 경치도 좋고\n잘쉬다 갑니다], [5.0, 6일...",이용할때마다 직원 친절 단지 객실내 먼지는 많야서 약간 아쉬워오ㅡ 시설도 좋고 경치...,NaN,"['이용', '직원', '친절', '실내', '먼지', '많야서', '아쉬워오', ...",[],"{'청결': '0.53%', '시설': '0.41%', '객실': '0.18%', ...",36.177165,127.157449,"{'청결': 9, '시설': 7, '객실': 3, '이용': 4, '직원': 2, ..."
1,충남,당진 큐비스트 호텔,"[미니바, 와이파이, 욕실용품, 금연, TV, 엘레베이터, 에어컨, 프린터사용, 짐...",충남 당진시 송악읍 부곡리 143-16,"[[5.0, 12일 전, 들어가자마자 새가구 냄새가 확 났어요ㅎㅎ\n다 깨끗하고,에...",들어가자마자 새가구 냄새가 확 났어요ㅎㅎ다 청결 무풍에 공기청정기까지 있어서 너무 ...,우선 신축 건물인지 외부내부는 깔끔합니다 진짜 너무 최악인건 샴푸 바디워시 컨디셔너...,"['가구', '냄새', '청결', '무풍', '공기청정기', '만족합니다', '미세...","['신축', '건물', '외부', '내부', '깔끔합니다', '최악', '샴푸', ...","{'청결': '1.06%', '시설': '0.17%', '객실': '0.19%', ...",36.948582,126.777809,"{'청결': 49, '시설': 8, '객실': 9, '이용': 13, '직원': 7..."
2,충남,어반스테이 당진터미널,"[와이파이, 욕실용품, 금연, TV, 객실내취사, 엘레베이터, 에어컨, 냉장고, 짐...",충남 당진시 수청동 1226,"[[5.0, 15시간 전, 당진에 최첨단 숙소가 들어왔네요\n일단 깨끗하고 전자레인...",당진에 최첨단 숙소가 들어왔네요일단 청결 전자레인지 싱크대 등 직접 요리해 먹을 수...,NaN,"['당진', '최첨단', '청결', '전자레인지', '싱크대', '직접', '요리'...",[],"{'청결': '0.81%', '시설': '0.10%', '객실': '0.70%', ...",36.899753,126.646858,"{'청결': 50, '시설': 6, '객실': 43, '이용': 27, '주변': ..."
3,충남,보령 호텔 그랜드 베이,"[와이파이, 욕실용품, 금연, TV, 엘레베이터, 에어컨, 냉장고, 짐보관가능, 객...",충남 보령시 궁촌동 342-3,"[[5.0, 12시간 전, 좋았어요 추천합니다 !\n(근데 아고다 예약이 더 싸요 ...",좋았어요 추천합니다 근데 아고다 예약이 더 싸요 ㅋ 청결 청결 만족합니다 3식구지만...,NaN,"['추천', '아고다', '예약', '청결', '청결', '만족합니다', '식구',...",[],"{'청결': '0.76%', '시설': '0.16%', '객실': '0.23%', ...",36.343454,126.589201,"{'청결': 47, '시설': 10, '객실': 14, '이용': 14, '직원':..."
4,충남,대천 버니프렌즈호텔,"[와이파이, 욕실용품, 금연, TV, 엘레베이터, 에어컨, 프린터사용, 라운지, 냉...",충남 보령시 남곡동 811-11,"[[5.0, 3개월 전, 친절하고 대천ic및 해수욕장, 보령 시내,\n기타 관광지와...",키즈 전용 호텔이라 주말엔 비싸지만 평일 이용하면 가격 청결 환한 호텔 이용하는 장...,신축이고 아기들을 위한 시설이 잘 갖춰졌다 생각하고 방문하였습니다12월 방문한 사람...,"['키즈', '전용', '주말', '비싸지만', '평일', '이용', '가격', '...","['신축', '아기', '시설', '생각', '사람', '개월', '이하', '아기...","{'청결': '0.27%', '시설': '0.27%', '객실': '0.27%', ...",36.348925,126.556420,"{'청결': 8, '시설': 8, '객실': 8, '이용': 10, '직원': 9,..."
5,충남,토모노야 호텔&료칸 대천,"[조식포함, 와이파이, 욕실용품, 레스토랑, 금연, TV, 에어컨, 냉장고, 짐보관...",충남 보령시 남곡동 811-5,"[[5.0, 9일 전, 꼭 가보고 싶었어서 예약해서 왔는데 잘 쉬다 갑니당~~^^ ...",밥이 나와요 맛있어요 넓어요조용해요 탕도 넓어요 꼭 가보고 싶었어서 예약해서 왔는데...,다들 도망가세요 돈이 너무 아까워요 객실도 눅눅하고 조식 석식 너무너무 기대했는데 ...,"['맛있어요', '넓어요', '조용해요', '넓어요', '예약', '별로', '괜찮...","['아까워요', '객실', '눅눅하고', '조식', '석식', '기대했는데', '역...","{'청결': '0.16%', '시설': '0.06%', '객실': '0.13%', ...",36.348827,126.556072,"{'청결': 11, '시설': 4, '객실': 9, '이용': 9, '직원': 7,..."
6,충남,호텔 마스타 대천,"[와이파이, 욕실용품, 금연, TV, 엘레베이터, 에어컨, 냉장고, 짐보관가능, 객...",충남 보령시 신흑동 1945,"[[5.0, 1개월 전, 바다랑가깝고 깔끔하며 배드가 넓어서 좋았습니다ㅎㅎ], [5...",낡았으나 주변 접근성 좋고 자는데 불편함 없음 낡았으나 주변 접근성 좋고 자는데 불...,진짜 다시는 이용하고 싶지 않읏 곳이네요청소상태도 완전 엉망에다가 매트리스 너무 안...,"['낡았으나', '주변', '접근성', '불편함', '낡았으나', '주변', '접근...","['이용', '않읏', '요청', '상태', '완전', '엉망', '매트리스', '...","{'청결': '0.18%', '시설': '0.37%', '객실': '0.37%', ...",36.307068,126.518446,"{'청결': 2, '시설': 4, '객실': 4, '이용': 2, '직원': 2, ..."
7,충남,한화리조트 대천 파로스,"[와이파이, 레스토랑, 금연, TV, 엘레베이터, 냉장고, 객실샤워실, BBQ, 카...",충남 보령시 신흑동 2017,"[[5.0, 4일 전, 직원분들 친절하시고 깨끗하고 청결해서다시 오고싶은 곳입니다]...",청결 바다 근처라 산책하기 좋았어요 사우나도 잘 되어 있습니다 태블릿으로 필요한 요...,친구들과 단체로 가서 자고오기 좋아요객실 깨끗한건 기대히면 안될 것 같고요조금만 더...,"['청결', '바다', '근처', '산책', '사우나', '태블릿', '필요한', ...","['친구', '단체', '자고', '오기', '객실', '깨끗한건', '기대히면',...","{'청결': '0.52%', '시설': '0.30%', '객실': '0.14%', ...",36.304992,126.519745,"{'청결': 33, '시설': 19, '객실': 9, '이용': 26, '직원': ..."
8,충남,데이원스테이 호텔,"[와이파이, 욕실용품, 금연, TV, 엘레베이터, 에어컨, 냉장고, 객실샤워실, 드...",충남 보령시 신흑동 2214-4,"[[5.0, 16일 전, 새로 지은 호텔이라 깨끗하고 대천비치와 아주 가까워 좋아요...",새로 지은 호텔이라 청결 대천비치와 아주 가까워 좋아요👍👍👍 청결 청결 청결 3박자...,NaN,"['새로', '지은', '청결', '비치', '가까워', '청결', '청결', '청...",[],"{'청결': '0.71%', '시설': '0.09%', '객실': '0.09%', ...",36.320313,126.508682,"{'청결': 16, '시설': 2, '객실': 2, '직원': 2, '가격': 13..."
9,충남,대천 323콘도,"[와이파이, 욕실용품, 레스토랑, 금연, TV, 에어컨, 

In [33]:
type(cnam['good_keyword'][1])

list

In [34]:
cnam.to_csv('충남1.csv',index=False)
one.to_csv('강원1.csv',index=False)
gg.to_csv('경기1.csv',index=False)
ic.to_csv('인천1.csv',index=False)
cbook.to_csv('충북1.csv',index=False)
jeju.to_csv('제주도1.csv',index=False)
seoul.to_csv('서울1.csv',index=False)
gnam.to_csv('경남1.csv',index=False)
gbook.to_csv('경북1.csv',index=False)
daegu.to_csv('대구1.csv',index=False)
daej.to_csv('대전1.csv',index=False)
busan.to_csv('부산1.csv',index=False)
sejong.to_csv('세종1.csv',index=False)
ulsan.to_csv('울산1.csv',index=False)
jnam.to_csv('전남1.csv',index=False)
jbook.to_csv('전북1.csv',index=False)
gj.to_csv('광주1.csv',index=False)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## 키워드를 입력하면 해당 키워드의 상위 n개의 데이터 추출

In [10]:
# TOP10긍정 키워드 비율 딕셔너리에서 키워드 선택하면 그 키워드 topn 정렬 후 추출
def ratio_topn(df,keyword,n):
# 'good_keyword_ratio' 열의 각 행에 있는 딕셔너리를 기반으로 keyword 키의 값을 추출하여 리스트에 저장
    clean_list = []
    for i in range(len(df['good_keyword_ratio'])):
        dict = df.at[i, 'good_keyword_ratio']
        if keyword in dict:
            clean_list.append(dict[keyword])
        else:  # keyword 키가 없으면 기본값으로 0을 추가
            clean_list.append(0)  
    # 데이터프레임을 정렬
    df_sorted = df.iloc[pd.Series(clean_list).sort_values(ascending=False).index][:n]
    return df_sorted

In [11]:
# TOP10긍정 키워드 빈도수 딕셔너리에서 키워드 선택하면 그 키워드 topn 정렬 후 추출
def freq_topn(df,keyword,n):
    clean_list = []
    for i in range(len(df['good_keyword_freq'])):
        dict = df.at[i, 'good_keyword_freq']
        if keyword in dict:
            clean_list.append(dict[keyword])
        else:  # keyword 키가 없으면 기본값으로 0을 추가
            clean_list.append(0)  
    # 데이터프레임을 정렬
    df_sorted = df.iloc[pd.Series(clean_list).sort_values(ascending=False).index][:n]
    return df_sorted

In [12]:
# TOPn 리뷰 추출
def review_topn(df,hotel_name, n):
    for i in range(len(df)):
        reviews = df[df['hotel_name'] == hotel_name]['star&reviews'].tolist()
        sorted_reviews = sorted(reviews, key=lambda x: x[0], reverse=True)
        result = sorted_reviews[0][:n]
    return result

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## 호텔 클릭시 -> 호텔명 / 시설정보 / 리뷰 n개 추출

In [13]:
def click_hotel(df,hotel_name,n):
    hotel = df[df['hotel_name'] == hotel_name]['hotel_name'].iloc[0]
    services = df[df['hotel_name'] == hotel_name]['services'].iloc[0]
    review = review_topn(df,hotel_name,n)
    return hotel, services, review

## 키워드 빈도수로 할 경우

In [32]:
one_sor_freq = freq_topn(one,'청결',10)
one_sor_freq['hotel_name'].tolist()

['더 베네치아 스위트',
 '속초 비즈니스 호텔 카멜',
 '센텀마크 호텔 양양',
 '르부르낙산 바이 체스터톤스',
 '강릉씨티호텔',
 '어반스테이 속초등대해변',
 '속초 아이파크스위트 호텔앤레지던스',
 '청초호베스트호텔',
 '어반스테이 속초해변 AB',
 '하이오션 경포']

In [15]:
click_hotel(one_sor_freq,'속초 비즈니스 호텔 카멜',3)

('속초 비즈니스 호텔 카멜',
 ['조식포함',
  '와이파이',
  '욕실용품',
  '금연',
  'TV',
  '엘레베이터',
  '에어컨',
  '냉장고',
  '짐보관가능',
  '객실샤워실',
  '무료주차',
  '카페',
  '드라이기',
  '카드결제',
  '주차장'],
 [[5.0, '3일 전', '깨끗하고 항상 친절합니다'],
  [5.0, '5일 전', '방도 깔끔하고 수압이나 소음 하나도 불편한게 없었습니다!'],
  [5.0, '6일 전', '역시 믿고 보는 숙소~\n짱입니다^^']])

## 키워드 비율로 할 경우

In [31]:
one_sor_ratio = ratio_topn(one,'청결',10)
one_sor_ratio['hotel_name'].tolist()

['속초 비즈니스 호텔 카멜',
 '더 베네치아 스위트',
 '센텀마크 호텔 양양',
 '강릉씨티호텔',
 '동해오션시티 레지던스호텔',
 '청초호베스트호텔',
 '라마다 속초',
 '오션투유리조트 설악비치 호텔&콘도',
 '속초 아이파크스위트 호텔앤레지던스',
 '하이원 그랜드 호텔 메인타워']

In [17]:
hotel, services, review = click_hotel(one_sor_ratio,'속초 비즈니스 호텔 카멜',3)

In [18]:
hotel

'속초 비즈니스 호텔 카멜'

In [19]:
services

['조식포함',
 '와이파이',
 '욕실용품',
 '금연',
 'TV',
 '엘레베이터',
 '에어컨',
 '냉장고',
 '짐보관가능',
 '객실샤워실',
 '무료주차',
 '카페',
 '드라이기',
 '카드결제',
 '주차장']

In [20]:
review

[[5.0, '3일 전', '깨끗하고 항상 친절합니다'],
 [5.0, '5일 전', '방도 깔끔하고 수압이나 소음 하나도 불편한게 없었습니다!'],
 [5.0, '6일 전', '역시 믿고 보는 숙소~\n짱입니다^^']]